In [241]:
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Team
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
import pandas as pd
import numpy as np
from basketball_reference_scraper.box_scores import get_box_scores
from basketball_reference_scraper.seasons import get_schedule, get_standings

In [242]:
team_abbreviations = {'ATLANTA HAWKS' : 'ATL'
,'BOSTON CELTICS' : 'BOS'
,'BROOKLYN NETS' : 'BRK'
,'CHICAGO BULLS' : 'CHI'
,'CHARLOTTE HORNETS' : 'CHO'
,'CLEVELAND CAVALIERS' : 'CLE'
,'DALLAS MAVERICKS' : 'DAL'
,'DENVER NUGGETS' : 'DEN'
,'DETROIT PISTONS' : 'DET'
,'GOLDEN STATE WARRIORS' : 'GSW'
,'HOUSTON ROCKETS' : 'HOU'
,'INDIANA PACERS' : 'IND'
,'LOS ANGELES CLIPPERS' : 'LAC'
,'LOS ANGELES LAKERS' : 'LAL'
,'MEMPHIS GRIZZLIES' : 'MEM'
,'MIAMI HEAT' : 'MIA'
,'MILWAUKEE BUCKS' : 'MIL'
,'MINNESOTA TIMBERWOLVES' : 'MIN'
,'NEW ORLEANS PELICANS' : 'NOP'
,'NEW YORK KNICKS' : 'NYK'
,'OKLAHOMA CITY THUNDER' : 'OKC'
,'ORLANDO MAGIC' : 'ORL'
,'PHILADELPHIA 76ERS' : 'PHI'
,'PHOENIX SUNS' : 'PHO'
,'PORTLAND TRAIL BLAZERS' : 'POR'
,'SACRAMENTO KINGS' : 'SAC'
,'SAN ANTONIO SPURS' : 'SAS'
,'TORONTO RAPTORS' : 'TOR'
,'UTAH JAZZ' : 'UTA'
,'WASHINGTON WIZARDS' : 'WAS'}

In [254]:
#Sample output from get_box_scores fuction

d = get_box_scores('2018-10-16', 'OKC', 'GSW', period='GAME', stat_type='ADVANCED')
d

{'OKC':                       PLAYER            MP           TS%          eFG%  \
 0                Paul George         37:24          .509          .478   
 1               Steven Adams         35:36          .548          .500   
 2           Dennis SchrÃ¶der         34:05          .485          .421   
 3          Terrance Ferguson         26:31          .000          .000   
 4          Patrick Patterson         18:53          .339          .278   
 5               Jerami Grant         29:07          .400          .357   
 6              Ãlex Abrines         23:28          .500          .500   
 7             Raymond Felton         13:55          .417          .200   
 8               Nerlens Noel         12:24          .521          .500   
 9             Hamidou Diallo          8:37          .450          .500   
 10               Abdel Nader  Did Not Play  Did Not Play  Did Not Play   
 11             Deonte Burton  Did Not Play  Did Not Play  Did Not Play   
 12  TimothÃ© Luwa

In [231]:
key, value = list(d.items())[0]
value[-1:]

,PLAYER,MP,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
13,Team Totals,240,.448,.420,.299,.264,12.2,77.4,46.1,52.9,7.7,8.3,14.1,100.0,83.4,100.7,NaN


In [261]:
schedule_df = get_schedule(2019, playoffs=False)
schedule_df.loc[schedule_df['DATE'] == '2019-02-22']
#schedule_df 

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
872,2019-02-22,Washington Wizards,110,Charlotte Hornets,123
873,2019-02-22,New Orleans Pelicans,111,Indiana Pacers,126
874,2019-02-22,Chicago Bulls,110,Orlando Magic,109
875,2019-02-22,San Antonio Spurs,117,Toronto Raptors,120
876,2019-02-22,Detroit Pistons,125,Atlanta Hawks,122
877,2019-02-22,Minnesota Timberwolves,115,New York Knicks,104
878,2019-02-22,Los Angeles Clippers,112,Memphis Grizzlies,106
879,2019-02-22,Denver Nuggets,114,Dallas Mavericks,104
880,2019-02-22,Utah Jazz,147,Oklahoma City Thunder,148


I'll have to break out the Home and Home Points column into another row. So each game should be worth 2 rows, one with each team, their points, and then their respective advanced box scores

In [259]:
#TS%	eFG%	3PAr	FTr	ORB%	DRB%	TRB%	AST%	STL%	BLK%	TOV%	USG%	ORtg	DRtg
final_schedule_df = pd.DataFrame(columns=['Date', 'Team', 'Points','TS%','eFG%','3PAr','FTr','ORB%',
                                          'DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','ORtg','DRtg'])

for index, row in schedule_df.iterrows():
    
    if index == 877:
        continue
        
    game_date = str(schedule_df['DATE'][index]).strip()[0:10]
    v_score = schedule_df['VISITOR_PTS'][index]
    h_score = schedule_df['HOME_PTS'][index]
    visitor = team_abbreviations.get(str(schedule_df['VISITOR'][index]).upper().strip())
    home = team_abbreviations.get(str(schedule_df['HOME'][index]).upper().strip())
    
    b = get_box_scores(game_date, visitor, home ,period='GAME', stat_type='ADVANCED')
    
    v_key, v_value = list(b.items())[0]
    v_TS = str(v_value[-1:]['TS%'])[6:10]
    v_eFG = str(v_value[-1:]['eFG%'])[6:10]
    v_3PAr = str(v_value[-1:]['3PAr'])[6:10]
    v_FTr = str(v_value[-1:]['FTr'])[6:10]
    v_ORB = str(v_value[-1:]['ORB%'])[6:10].rstrip()
    v_DRB = str(v_value[-1:]['DRB%'])[6:10].rstrip()
    v_TRB = str(v_value[-1:]['TRB%'])[6:10].rstrip()
    v_AST = str(v_value[-1:]['AST%'])[6:10].rstrip()
    v_STL = str(v_value[-1:]['STL%'])[6:10].rstrip()
    v_BLK = str(v_value[-1:]['BLK%'])[6:10].rstrip()
    v_TOV = str(v_value[-1:]['TOV%'])[6:10].rstrip()
    v_USG = str(v_value[-1:]['USG%'])[6:11].rstrip()
    v_ORtg = str(v_value[-1:]['ORtg'])[6:11].rstrip()
    v_DRtg = str(v_value[-1:]['DRtg'])[6:11].rstrip()
    
    
    final_schedule_df = final_schedule_df.append({'Date': game_date,'Team':visitor,'Points': v_score,'TS%': v_TS,'eFG%':v_eFG
                                                 ,'3PAr':v_3PAr,'FTr':v_FTr,'ORB%':v_ORB,'DRB%':v_DRB,'TRB%':v_TRB,'AST%':v_AST
                                                 ,'STL%':v_STL,'BLK%':v_BLK,'TOV%':v_TOV,'USG%':v_USG,'ORtg':v_ORtg,'DRtg':v_DRtg}, ignore_index=True)
    
    h_key, h_value = list(b.items())[1]
    h_TS = str(h_value[-1:]['TS%'])[6:10]
    h_eFG = str(h_value[-1:]['eFG%'])[6:10]
    h_3PAr = str(h_value[-1:]['3PAr'])[6:10]
    h_FTr = str(h_value[-1:]['FTr'])[6:10]
    h_ORB = str(h_value[-1:]['ORB%'])[6:10].rstrip()
    h_DRB = str(h_value[-1:]['DRB%'])[6:10].rstrip()
    h_TRB = str(h_value[-1:]['TRB%'])[6:10].rstrip()
    h_AST = str(h_value[-1:]['AST%'])[6:10].rstrip()
    h_STL = str(h_value[-1:]['STL%'])[6:10].rstrip()
    h_BLK = str(h_value[-1:]['BLK%'])[6:10].rstrip()
    h_TOV = str(h_value[-1:]['TOV%'])[6:10].rstrip()
    h_USG = str(h_value[-1:]['USG%'])[6:11].rstrip()
    h_ORtg = str(h_value[-1:]['ORtg'])[6:11].rstrip()
    h_DRtg = str(h_value[-1:]['DRtg'])[6:11].rstrip()
    
    final_schedule_df = final_schedule_df.append({'Date': game_date,'Team':home,'Points': h_score,'TS%': h_TS,'eFG%':h_eFG
                                                 ,'3PAr':h_3PAr,'FTr':h_FTr,'ORB%':h_ORB,'DRB%':h_DRB,'TRB%':h_TRB,'AST%':h_AST
                                                 ,'STL%':h_STL,'BLK%':h_BLK,'TOV%':h_TOV,'USG%':h_USG,'ORtg':h_ORtg,'DRtg':h_DRtg}, ignore_index=True)
    #final_schedule_df = final_schedule_df.append({'Date': game_date,'Team':home,'Points': h_score, 'TS%': h_TS}, ignore_index=True)

In [260]:
final_schedule_df.to_csv('NBA Project.csv',index=False)